In [1]:
import time
import os
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
browser = webdriver.Chrome()
url = ["https://medium.com/topic/data-science", "https://medium.com/topic/artificial-intelligence",
       "https://medium.com/topic/technology", "https://medium.com/topic/programming"]
browser.get("https://medium.com/topic/data-science")
time.sleep(1)
import ast
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

### Fetch all links on that page and filter them out

In [2]:
# import pickle
# with open('links.pkl', 'r') as f:
#     all_posts = pickle.load(f)

### Iterate over every link and fetch relevant data

In [16]:
data_scrapped = pd.read_csv("data_scraped.csv")
del data_scrapped['index']
data_scrapped.shape

(4824, 11)

In [17]:
data_scrapped['datePublished'] = data_scrapped['datePublished'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))

In [27]:
data_scrapped.drop_duplicates().shape

(4712, 11)

In [21]:
data_scraped.head(3)

,title,claps,datePublished,read_time,followedBy,following,para,tags,bullets,images,links
0,Python is the Perfect Tool for any Problem,6.4K,2018-02-04,3,1497,84,"[Reflecting on my first Python program, Reflec...","[Python, Programming, Education, Data Science,...",[The best way to learn a new skill is to find ...,"[<img class=""graf-image"" data-action=""zoom"" da...","[https://automatetheboringstuff.com/, https://..."
1,Computer Science VS Software Engineering — Which,2.1K,2018-02-06,7,1651,62,"[Hey everyone! My name is YK, and I’m currentl...","[Programming, Tech, Careers, Startup, Self Imp...",[Computer Science is the study of how computer...,"[<img class=""graf-image"" data-action=""zoom"" da...","[https://www.youtube.com/csdojo, https://mediu..."
2,It’s a no-brainer! Deep learn,4K,2018-02-05,12,180,139,"[Written by Atli Kosson and Henrik Marklund, F...","[Machine Learning, Deep Learning, Healthcare, ...","[The segmentation problem, The data from MR sc...","[<img class=""graf-image"" data-action=""zoom"" da...","[http://cs231n.stanford.edu/, https://arxiv.or..."


In [23]:
data = pd.DataFrame()
for i in range(data_scrapped.shape[0]):
    
    data.loc[i, "title_words"] = len(data_scrapped.loc[i, "title"].split(" "))
    data.loc[i, "days_passed"] = (datetime.now() - data_scrapped.loc[i,'datePublished'] ).days
    data.loc[i, "ct_image"] = len(data_scrapped.loc[i, "images"])
    data.loc[i, "ct_tags"] = len(data_scrapped.loc[i, "tags"])
    data.loc[i, "text"] =  "".join(data_scrapped.loc[i, "para"] + data_scrapped.loc[i, "bullets"])
    data.loc[i, "ct_words"] = len(data.loc[i, "text"].split())
    data.loc[i, "title_emot_quotient"] = abs(sid.polarity_scores(data_scrapped.loc[i, "title"])['compound'])
    data.loc[i, "featured_in_tds"] = 'Towards Data Science' in data_scrapped.loc[i,'tags']
    data.loc[i,'read_time'] = int(data_scrapped.loc[i,'read_time']) 
    data.loc[i,'links'] =  data_scrapped.loc[i,'links']
    data.loc[i,'following'] = int(data_scrapped.loc[i,'following']) 
    data.loc[i,'followedBy'] = int(data_scrapped.loc[i,'followedBy'])

    
def claps2num(x):
    if "K" in x:
        return float(x.replace("K",""))*1000
    else:
        return int(x)
data['claps'] = data_scrapped['claps'].apply(lambda x: claps2num(x))
data["img/word"] = data['ct_image']/data['ct_words']

In [24]:
data.shape

(4824, 14)

In [25]:
data.head(3)

,title_words,days_passed,ct_image,ct_tags,text,ct_words,title_emot_quotient,featured_in_tds,read_time,links,following,followedBy,claps,img/word
0,8.0,5.0,475.0,68.0,"[Reflecting on my first Python program, Reflec...",658.0,0.25,True,3.0,"[https://automatetheboringstuff.com/, https://...",84.0,1497.0,6400.0,0.721884
1,7.0,3.0,255.0,55.0,"[Hey everyone! My name is YK, and I’m currentl...",1481.0,0.00,False,7.0,"[https://www.youtube.com/csdojo, https://mediu...",62.0,1651.0,2100.0,0.172181
2,5.0,4.0,3666.0,66.0,"[Written by Atli Kosson and Henrik Marklund, F...",2673.0,0.00,False,12.0,"[http://cs231n.stanford.edu/, https://arxiv.or...",139.0,180.0,4000.0,1.371493


In [26]:
data.drop_duplicates().shape

(4712, 14)

In [132]:
data.to_csv("data.csv",index=False, encoding='utf-8')

In [131]:
data_scrapped.head()

,title,claps,datePublished,read_time,para,tags,bullets,images
0,Python is the Perfect Tool for any Problem,6.3K,2018-02-04,3,"[Reflecting on my first Python program, Reflec...","[Python, Programming, Education, Data Science,...",[The best way to learn a new skill is to find ...,"[<img class=""graf-image"" data-action=""zoom"" da..."
1,Computer Science VS Software Engineering — Which,2K,2018-02-06,7,"[Hey everyone! My name is YK, and I’m currentl...","[Programming, Tech, Careers, Startup, Self Imp...",[Computer Science is the study of how computer...,"[<img class=""graf-image"" data-action=""zoom"" da..."
2,It’s a no-brainer! Deep learn,3.9K,2018-02-05,12,"[Written by Atli Kosson and Henrik Marklund, F...","[Machine Learning, Deep Learning, Healthcare, ...","[The segmentation problem, The data from MR sc...","[<img class=""graf-image"" data-action=""zoom"" da..."
3,The 5 Clustering Algorithms Data,1.8K,2018-02-05,11,[Clustering is a Machine Learning technique th...,"[Machine Learning, Clustering, Data Science, A...","[To begin, we first select a number of classes...","[<img class=""graf-image"" data-height=""480"" dat..."
4,Building Prediction APIs in Python (Part 2,53,2018-02-04,10,"[In the initial series post, we created a simp...","[Machine Learning, Python, API, Scikit Learn]",[Online Advertising: Suppose we’ve built a mod...,"[<img class=""graf-image"" data-height=""426"" dat..."


In [21]:
r  = requests.get("https://medium.com/@neerja.doshi/augmentation-for-image-classification-24ffcbc38833")
data = r.text
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
print soup.prettify()

In [32]:
soup.find_all("script")[-1].text.split('"usersFollowedCount":')[1].split(",")[0]

u'14'

In [43]:
soup.find_all('a', class_ = "markup--anchor markup--p-anchor")[1]['href']

u'https://github.com/fastai/fastai.git'

In [4]:
# import pickle
# with open('links.pkl', 'wb') as f:
#     pickle.dump(all_posts, f,protocol=2)

In [ ]:
# data_scrapped.drop_duplicates(inplace=True)
# data_scrapped.to_csv('data_scraped.csv', index = False, encoding='utf-8')

In [44]:
# para = []
# tags = []
# bullets = []
# images = []
# links = []
# data_scrapped = pd.DataFrame()

In [46]:
# data_scrapped.dropna(inplace=True)
# data_scrapped.reset_index(inplace=True)

In [47]:
# data_scrapped['para'] = [[]] * len(data_scrapped)
# data_scrapped['tags'] = [[]] * len(data_scrapped)
# data_scrapped['bullets'] = [[]] * len(data_scrapped)
# data_scrapped['images'] = [[]] * len(data_scrapped)
# data_scrapped['links'] = [[]] * len(data_scrapped)

In [48]:
# len(para), len(tags), len(bullets), len(images), len(links), data_scrapped.shape

(4824, 4824, 4824, 4824, 4824, (4824, 12))

In [49]:
# data_scrapped['tags'] = tags
# data_scrapped['para'] = para
# data_scrapped['bullets'] = bullets
# data_scrapped['images'] = images
# data_scrapped['links'] = links

In [52]:
# del data_scrapped['index']
# data_scrapped.shape

(4824, 12)

In [ ]:
# for i in range(len(all_posts)):

#     try:
#         r  = requests.get(all_posts[i])
#         data = r.text
#         soup = BeautifulSoup(data, 'html.parser')
#         data_scrapped.loc[i,'title'] = soup.title.text[0:-23]
#         data_scrapped.loc[i,'claps'] = soup.find('button', class_="button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton").text
#         format = "%Y-%m-%d"
#         data_scrapped.loc[i,'datePublished'] =  datetime.strptime(ast.literal_eval(soup.find('script').text)["datePublished"][0:10], format)
#         data_scrapped.loc[i,'read_time'] = soup.find("span",class_ = "readingTime")['title'][:-9]
#         data_scrapped.loc[i,'followedBy'] = soup.find_all("script")[-1].text.split('"usersFollowedByCount":')[1].split(",")[0]
#         data_scrapped.loc[i,'following'] = soup.find_all("script")[-1].text.split('"usersFollowedCount":')[1].split(",")[0]
        
#         links.append([item['href'] for item in soup.find_all('a', class_ = "markup--anchor markup--p-anchor")])
#         para.append( [hit.text.strip() for hit in soup.find_all('p')])
#         tags.append( ast.literal_eval(soup.find('script').text)["keywords"])
#         bullets.append( [hit.text.strip() for hit in soup.select('li[class*="graf graf--li"]')])
#         images.append( soup.find_all('img', class_="graf-image"))
#     except:
#         print(i)
#     if i%500 == 0:
#         print(i)

In [3]:
# all_posts = []
# for u in url:
    
#     browser.get(u)
#     time.sleep(1)
    
#     elem = browser.find_element_by_tag_name('body')
#     # print(elem)

#     no_of_pagedowns = 3000

#     while no_of_pagedowns:
#         elem.send_keys(Keys.PAGE_DOWN)
#         time.sleep(0.2)
#         no_of_pagedowns-=1


#     post_elems = browser.find_elements_by_tag_name("a")

#     # for post in post_elems:
#     #     print(post.get_attribute('href'), len (post.get_attribute('href')))
#     post_relevant = [post for post in post_elems if len(post.get_attribute('href')) > 100]  
#     for i in range(0, len(post_relevant), 3):
#         all_posts.append(post_relevant[i].get_attribute('href'))
#     len(all_posts)